In [1]:
from pytorch_lightning.loggers import WandbLogger

import wandb

from FNN import *

wandb.login()

wandb: Currently logged in as: bhavye-mathur. Use `wandb login --relogin` to force relogin


True

In [2]:
BATCH_SIZE = 64

WindDataset.init()

train_data = WindDataset("train")
validation_data = WindDataset("validation")
test_data = WindDataset("test")

WindModel.init(BATCH_SIZE, train_data, validation_data, test_data)

del WindDataset.data

In [3]:
VARIABLE = "U"
INPUT_SIZE = 15
OUTPUT_SIZE = 2
HIDDEN_SIZES = [256, 128]
LEARNING_RATE = 1e-3

ACTIVATION = torch.nn.ReLU
LOSS_FUNC = torch.nn.functional.mse_loss

model = WindModel("U", LEARNING_RATE, LOSS_FUNC, INPUT_SIZE, HIDDEN_SIZES, OUTPUT_SIZE, ACTIVATION)
model


WindModel(
  (model): Sequential(
    (0): Linear(in_features=15, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=128, bias=True)
    (3): ReLU()
    (4): Linear(in_features=128, out_features=2, bias=True)
  )
)

In [4]:
wandb_logger = WandbLogger(project="MERRA2-Wind", log_model="all")
wandb_logger.experiment.config["estimate_quantile"] = ESTIMATE_QUANTILE


In [5]:
trainer = pl.Trainer(devices=1,
                     accelerator="mps",
                     # precision=16,

                     # limit_train_batches=0.1,
                     # limit_val_batches=0.1,
                     # val_check_interval=0.25,
                     #
                     auto_lr_find=True,
                     max_epochs=1,
                     #
                     # logger=wandb_logger
                     )

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/bhavyemathur/Desktop/Projects/Spherindrical Fourier Transform/MERRA-2/venv/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:67: UserWarning: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
  warning_cache.warn(


In [ ]:
%matplotlib notebook
%env PYTORCH_ENABLE_MPS_FALLBACK=1

lr_finder = trainer.tuner.lr_find(model, max_lr=1e-1)
lr_finder.plot(suggest=True, show=True)

/Users/bhavyemathur/Desktop/Projects/Spherindrical Fourier Transform/MERRA-2/venv/lib/python3.10/site-packages/pytorch_lightning/loops/utilities.py:94: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(
Missing logger folder: /Users/bhavyemathur/Desktop/Projects/Spherindrical Fourier Transform/MERRA-2/wind-prediction/models/lightning_logs
/Users/bhavyemathur/Desktop/Projects/Spherindrical Fourier Transform/MERRA-2/venv/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 10 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/Users/bhavyemathur/Desktop/Projects/Spherindrical Fourier Transform/MERRA-2/venv/lib/python3.10/site-p

env: PYTORCH_ENABLE_MPS_FALLBACK=1


In [ ]:
trainer.fit(model)


Exception in thread SystemMonitor:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/bhavyemathur/Desktop/Projects/Spherindrical Fourier Transform/MERRA-2/venv/lib/python3.10/site-packages/wandb/sdk/internal/system/system_monitor.py", line 118, in _start
    asset.start()
  File "/Users/bhavyemathur/Desktop/Projects/Spherindrical Fourier Transform/MERRA-2/venv/lib/python3.10/site-packages/wandb/sdk/internal/system/assets/cpu.py", line 166, in start
    self.metrics_monitor.start()
  File "/Users/bhavyemathur/Desktop/Projects/Spherindrical Fourier Transform/MERRA-2/venv/lib/python3.10/site-packages/wandb/sdk/internal/system/assets/interfaces.py", line 168, in start
    logger.info(f"Started {self._process

Sanity Checking: 0it [00:00, ?it/s]

/Users/bhavyemathur/Desktop/Projects/Spherindrical Fourier Transform/MERRA-2/venv/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 10 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
